<a href="https://colab.research.google.com/github/andcore20/Fifteen_Hundred_Curso_Dados/blob/main/sql_azure_to_s3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql.functions import *
from datetime import date

In [ ]:
url = "jdbc:sqlserver://191.232.198.224:1433;databaseName=db-curso-eng-dados;trustServerCertificate=true"
user = "user_aluno_dados"
password = dbutils.secrets.get(scope = "scope-curso-eng-dados", key = "sql-server-user-pass")

In [ ]:
dicionario_dfname = {
    'Calendario':'[Proj.Calendario]',
    'Detranmensal':'[Proj.Detranmensal]',
    'Detransemanal_01':'[Proj.Detransemanal_01]',
    'Feriadosnacionais':'[Proj.Feriadosnacionais]',
    'Integracao':'[Proj.Integracao]',
    'Meta':'[Proj.Meta]'
    }

for df in dicionario_dfname:
    print(df)
    print(dicionario_dfname[df])


Calendario
[Proj.Calendario]
Detranmensal
[Proj.Detranmensal]
Detransemanal_01
[Proj.Detransemanal_01]
Feriadosnacionais
[Proj.Feriadosnacionais]
Integracao
[Proj.Integracao]
Meta
[Proj.Meta]


In [ ]:
df = {}
for i in dicionario_dfname:
    df[i] = (spark.read
    .format("jdbc")
    .option("url", url)
    .option("dbtable", dicionario_dfname[i])
    .option("user", user)
    .option("password", password)
    .load()
    )

In [ ]:
display(df)

{'Calendario': DataFrame[column1: date, column2: timestamp],
 'Detranmensal': DataFrame[mes: int, marca: string, modelo: string, versao: string, dia: int, ano: smallint, vin: string],
 'Detransemanal_01': DataFrame[mes: int, marca: string, modelo: string, versao: string, dia: int, ano: smallint, vin: string],
 'Feriadosnacionais': DataFrame[Data: date, Dia_da_Semana: string, Feriado: string],
 'Integracao': DataFrame[vin: string, Dealer: string],
 'Meta': DataFrame[dealer: string, meta: smallint]}

In [ ]:
s3_bucket = "uni1500-curso-dados-2305"
s3_path = "staging/Grupo 01/coreliano_andre/staging/"

for i in df:
    df[i].write\
    .format("csv")\
    .mode("overwrite")\
    .option("header","true")\
    .save(f"s3a://{s3_bucket}/{s3_path+i}")